In [1]:
!pip install -U -q PyDrive

In [2]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
 
 
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import tensorflow as tf


# link = 'https://drive.google.com/file/d/18wU1HxS8bZxmdGHD63g5AyTXC-jRT_Vj/view?usp=sharing'
 
import pandas as pd
 
df = pd.read_csv('/content/drive/MyDrive/Ofir & Niv/HIT/NLP-Project/ML data.csv', usecols=['lyrics', 'Mood'])
df

,lyrics,Mood
0,she alway follow me from town to town at least...,1
1,sometim i feel im gonna break down and cri so ...,0
2,i was work at the railroad on the train to ark...,1
3,hush hush sweet charlott charlott dont you cri...,1
4,i met him in a bar that was the first mistak o...,0
...,...,...
106457,as the tear roll down my face lost in memori t...,0
106458,i know you scare of that cockadoodledo dont wo...,1
106459,way out there in the wilder of love and pain u...,1
106460,it take an aw lot to get me down but it onli t...,1


In [5]:
import matplotlib.pyplot as plt
import os
import re
import shutil
import string
import tensorflow as tf

from tensorflow.keras import layers
from tensorflow.keras import losses

In [ ]:
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Input, Dense, LSTM, Embedding
from keras.layers import Dropout, Activation, Bidirectional, GlobalMaxPool1D
from keras.models import Sequential
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.preprocessing import text, sequence

target = df['Mood']
y = pd.get_dummies(target).values

# use keras to create a Tokenizer object
tokenizer = text.Tokenizer(num_words=20000)  # limit to the num_words most important ones
tokenizer.fit_on_texts(list(df['lyrics']))
tokenized_texts = tokenizer.texts_to_sequences(df['lyrics'])
X = sequence.pad_sequences(tokenized_texts, maxlen=100)

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
from nltk import word_tokenize

data = df['lyrics'].map(word_tokenize).values
total_vocabulary = set(word.lower() for ly in data for word in ly)  # set created from nested comprehension 

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# create my NN model
model = Sequential()

embedding_size = 128
model.add(Embedding(len(total_vocabulary), embedding_size))
model.add(LSTM(25, return_sequences=True))
model.add(GlobalMaxPool1D())
model.add(Dropout(0.5))
model.add(Dense(50, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(50, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))  # use 4 because we have 4 categories

In [ ]:
X_train.shape

(85169, 100)

In [ ]:
model.compile(loss='categorical_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])
model.summary() # check the shape

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 128)         12249216  
                                                                 
 lstm (LSTM)                 (None, None, 25)          15400     
                                                                 
 global_max_pooling1d (Globa  (None, 25)               0         
 lMaxPooling1D)                                                  
                                                                 
 dropout (Dropout)           (None, 25)                0         
                                                                 
 dense (Dense)               (None, 50)                1300      
                                                                 
 dropout_1 (Dropout)         (None, 50)                0         
                                                        

In [ ]:
model.fit(X_train, y_train, epochs=10, batch_size=128, validation_split=0.1)

Epoch 1/10
599/599 [==============================] - 138s 226ms/step - loss: 0.6685 - accuracy: 0.5923 - val_loss: 0.6468 - val_accuracy: 0.6353
Epoch 2/10
599/599 [==============================] - 135s 225ms/step - loss: 0.6340 - accuracy: 0.6520 - val_loss: 0.6396 - val_accuracy: 0.6400
Epoch 3/10
599/599 [==============================] - 135s 225ms/step - loss: 0.6080 - accuracy: 0.6747 - val_loss: 0.6431 - val_accuracy: 0.6326
Epoch 4/10
599/599 [==============================] - 134s 223ms/step - loss: 0.5802 - accuracy: 0.6958 - val_loss: 0.6625 - val_accuracy: 0.6129
Epoch 5/10
599/599 [==============================] - 135s 225ms/step - loss: 0.5530 - accuracy: 0.7144 - val_loss: 0.6668 - val_accuracy: 0.6172
Epoch 6/10
599/599 [==============================] - 134s 223ms/step - loss: 0.5288 - accuracy: 0.7315 - val_loss: 0.7093 - val_accuracy: 0.6151
Epoch 7/10
599/599 [==============================] - 134s 224ms/step - loss: 0.5040 - accuracy: 0.7458 - val_loss: 0.7795 -

KeyboardInterrupt: ignored

In [ ]:
"""from sklearn.model_selection import train_test_split

# Let's say we want to split the data in 80:10:10 for train:valid:test dataset
train_size=0.8

X = df['lyrics']
y = df['Mood']

# In the first step we will split the data in training and remaining dataset
X_train, X_rem, y_train, y_rem = train_test_split(X, y, train_size=0.8, random_state=42, shuffle = True)

# Now since we want the valid and test size to be equal (10% each of overall data). 
# we have to define valid_size=0.5 (that is 50% of remaining data)
test_size = 0.5
X_valid, X_test, y_valid, y_test = train_test_split(X_rem,y_rem, test_size=0.5)

print(X_train.shape), print(y_train.shape)
print(X_valid.shape), print(y_valid.shape)
print(X_test.shape), print(y_test.shape)"""

In [ ]:
"""# train X & y
train_text_ds_raw = tf.data.Dataset.from_tensor_slices(
            tf.cast(X_train.values, tf.string)
) 
train_cat_ds_raw = tf.data.Dataset.from_tensor_slices(
            tf.cast(y_train.values, tf.int32),

) 

# validation X & y
valid_text_ds_raw = tf.data.Dataset.from_tensor_slices(
            tf.cast(X_valid.values, tf.string)
) 
valid_cat_ds_raw = tf.data.Dataset.from_tensor_slices(
            tf.cast(y_valid.values, tf.int32),

) 

# test X & y
test_text_ds_raw = tf.data.Dataset.from_tensor_slices(
            tf.cast(X_test.values, tf.string)
) 
test_cat_ds_raw = tf.data.Dataset.from_tensor_slices(
            tf.cast(y_test.values, tf.int32),

)"""

In [ ]:
"""import numpy as np
import pandas as pd
import tensorflow as tf
tf.enable_eager_execution()
training_df: pd.DataFrame = pd.DataFrame(
data={
'feature1': np.random.rand(10),
'feature2': np.random.rand(10),
'feature3': np.random.rand(10),
'target': np.random.randint(0, 3, 10)
}
)
features = ['feature1', 'feature2', 'feature3']
print(training_df)
training_dataset = (
tf.data.Dataset.from_tensor_slices(
(
tf.cast(training_df[features].values, tf.float32),
tf.cast(training_df['target'].values, tf.int32)
)
)
)
for features_tensor, target_tensor in training_dataset:
print(f'features:{features_tensor} target:{target_tensor}')
"""
# dataset = (
# tf.data.Dataset.from_tensor_slices(
# (
# tf.cast(df['lyrics'].values, tf.string),
# tf.cast(df['Mood'].values, tf.int32)
# )
# )
# )


In [ ]:
"""BUFFER_SIZE = 10000
BATCH_SIZE = 64"""

In [ ]:
"""import tensorflow as tf

train_text_ds_raw = train_text_ds_raw.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_text_ds_raw = test_text_ds_raw.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
valid_text_ds_raw = valid_text_ds_raw.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)"""

In [ ]:
"""VOCAB_SIZE = 10000
# encoder = tf.keras.layers.TextVectorization(
#     max_tokens=VOCAB_SIZE)
# encoder.adapt(train_text_ds_raw = train_text_ds_raw.map(lambda text, label: text))"""

In [ ]:
"""tf.keras.layers.TextVectorization(
    max_tokens=None,
    standardize="lower_and_strip_punctuation",
    split="whitespace",
    ngrams=None,
    output_mode="int",
    output_sequence_length=None,
    pad_to_max_tokens=False,
    vocabulary=None
)"""

In [ ]:
"""from tensorflow.keras.layers import TextVectorization

encoder = tf.keras.layers.TextVectorization(max_tokens=VOCAB_SIZE)
encoder.adapt(train_text_ds_raw)"""

In [ ]:
"""layers = [
    encoder,
    tf.keras.layers.Embedding(
        input_dim=(128, (len(encoder.get_vocabulary())),
        output_dim=64,
        # Use masking to handle the variable sequence lengths
        mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(2)
]

model = tf.keras.Sequential(layers)"""

"""
model = tf.keras.Sequential([
  layers.Embedding(max_features + 1, embedding_dim),
  layers.Dropout(0.2),
  layers.GlobalAveragePooling1D(),
  layers.Dropout(0.2),
  layers.Dense(1)])

model.summary()
"""

"""
model = Sequential()
model.add(Embedding(n_unique_words, n_dim,
    input_length=max_review_length))
model.add(Flatten())
model.add(Dense(n_dense, activation='relu')) model.add(Dropout(dropout))
# model.add(Dense(n_dense, activation='relu'))
# model.add(Dropout(dropout))
model.add(Dense(1, activation='sigmoid'))
"""""

In [ ]:
"""model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])"""

In [ ]:
"""model.summary()"""

In [ ]:
"""history = model.fit(train_text_ds_raw, epochs=10,
                    validation_data=test_text_ds_raw,
                    batch_size=128)"""

In [ ]:
"""test_loss, test_acc = model.evaluate(test_dataset)

print('Test Loss:', test_loss)
print('Test Accuracy:', test_acc)"""

In [ ]:
"""plt.figure(figsize=(16, 8))
plt.subplot(1, 2, 1)
plot_graphs(history, 'accuracy')
plt.ylim(None, 1)
plt.subplot(1, 2, 2)
plot_graphs(history, 'loss')
plt.ylim(0, None)"""

In [ ]:
"""model.fit(x_train, y_train,
    batch_size=batch_size, epochs=epochs, verbose=1,
    validation_data=(x_valid, y_valid),
    callbacks=[modelcheckpoint])
"""